In [15]:
# IMPORTS
import pandas as pd 
import wandb 
import numpy as np
import matplotlib.pyplot as plt 
import os, sys
import wandb 
import time
import json
import matplotlib as mpl
from statsmodels.tsa.arima_model import ARIMA
from causal_ccm.causal_ccm import ccm
from sklearn.decomposition import PCA
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import grangercausalitytests
from scipy.spatial import distance
from scipy.stats import pearsonr
# from nonlincausality.nonlincausality import nonlincausalityARIMA, nonlincausalityMLP
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
CAUSAL_PAIRS = [("predator_0", "predator_1"), ("predator_1", "predator_0")]
DIMENSIONS = ["x", "y", "dx", "dy", "PCA_1", "PCA_2"]

In [16]:
# GLOBALS + wandb init
ENTITY = 'tpn'
PROJECT = 'wolf_sample'

api = wandb.Api()
num_runs = len(api.runs(path=f'{ENTITY}/{PROJECT}'))
finished_runs = api.runs(
    path=f'{ENTITY}/{PROJECT}', 
    filters={"state": "finished"})
print(f"""
    Entity: {ENTITY}
    Project: {PROJECT}
    Number of runs: {num_runs}
    Finished runs: {len(finished_runs)}
    """)
runs_summary = pd.DataFrame()
for run in finished_runs:
    config = run.config
    summary = run.summary
    if run.state != 'finished':
        print(f'Run {run.name} is not finished')
        continue
    time.sleep(0.01)
    metrics = pd.DataFrame([dict(summary)])
    metrics['algorithm_type'] = config['algorithm_type']
    metrics['run_id'] = run.name
    metrics['env_config'] = json.dumps(config['env_config'])
    # metrics['nprey'] = config['env_config']['nprey']
    # metrics['reward_lone'] = config['env_config']['reward_lone']
    # metrics['reward_team'] = config['env_config']['reward_team']
    metrics['config'] = json.dumps(config)
    if len(runs_summary) == 0:
        runs_summary = metrics
    else:
        runs_summary = pd.concat([runs_summary, metrics])

# for a given set of nrpey and rteam, plot training curves
# get new runs for the set of parameters 
def get_performance_df(nprey, rteam, algo):
    collated_performances_df = None
    struct_df = pd.DataFrame(
            [],
            index = np.arange(0, 50001, 500),
            columns = ['episode_reward', 'episode_length', 'episode_assists', 'episodes_total']
        ).fillna(0)
    subset = api.runs(
        path=f'{ENTITY}/{PROJECT}', 
        filters={
            "state": "finished",
            "config.algorithm_type": algo,
            "config.env_config.nprey": nprey,
            "config.env_config.reward_team": rteam
            })
    print(f"""
            For nprey={nprey}, rteam={rteam}, algo={algo}:
            {len(subset)} runs
            """)
    if len(subset) == 0:
        return None 
    # get the training curves for the all runs in the subset 
    for run in subset:
        run_name = run.name
        performance_df = struct_df.copy()
        history = pd.DataFrame(run._full_history())
        df1 = history.loc[:, ['episode_reward_mean', 'episode_len_mean', 'episode_assists_mean', 'episodes_total']].dropna()
        df1.columns = ['episode_reward', 'episode_length', 'episode_assists', 'episodes_total']
        for col in df1.columns:
            if col == 'epsides_total':
                continue
            performance_df[col] = np.interp(struct_df.index, df1['episodes_total'], df1[col])

        collated_performances_df = np.dstack([np.array(struct_df), np.array(performance_df)])\
        if collated_performances_df is None else\
            np.dstack([collated_performances_df, np.array(performance_df)])
    collated_performances_df = collated_performances_df[:, :, 1:]
    collated_performances_df = pd.DataFrame(
        collated_performances_df.mean(-1),
        index = struct_df.index,
        columns = struct_df.columns)
    collated_performances_df = collated_performances_df.rolling(5, axis=0).mean().dropna()
    return collated_performances_df


    Entity: tpn
    Project: wolf_sample
    Number of runs: 364
    Finished runs: 288
    


In [17]:
# TIME SERIES ANALYSIS
def get_correlation(X, Y):
    return [("correlation", round(np.corrcoef(X, Y)[0][1],4))]

def get_ccm(X, Y, lag):
    E = lag
    tau = 1
    corr, pval = ccm(list(X), list(Y), tau=tau, E=E).causality()
    if pd.isna(corr):
        corr = 0.0
        pval = -1.0
    return [("ccm_score", round(corr, 4)),  ("ccm_pval", round(pval, 4))]


# Returns the results from fitting an OLS model to the data 
def get_granger_linear(X, Y, lag):
    try:
        data = np.array([X, Y]).T.astype(np.float64)
        results = grangercausalitytests(data, maxlag=[lag], verbose=False)
        ssr_ftest = results[lag][0]["ssr_ftest"]
        chi_test = results[lag][0]["ssr_chi2test"]
        return [
        ("F-statistic", ssr_ftest[0]),
        ("F-p", ssr_ftest[1]),
        ("chi2", chi_test[0]),
        ("chi-p", chi_test[1]),
    ]
    except:
        return []

In [5]:
nprey = 5
algo_type = 'independent'
rteam = 1.25

def fetch_run_data(nprey, algo_type, rteam):

    # get all the runs 
    subset = api.runs(
        path=f'{ENTITY}/{PROJECT}', 
        filters={
            "state": "finished",
            "config.algorithm_type": algo_type,
            "config.env_config.nprey": nprey,
            "config.env_config.reward_team": rteam
            })

    if len(subset) == 0:
        return None 
    
    for run in subset:
        history_df = None 
        performance_df = None 
        for file in run.files():
            if "history" in file.name:
                history_df = pd.read_json(
                    file.download(exist_ok=True), orient='split'
                )
            if "performance" in file.name:
                performance_df = pd.read_json(
                    file.download(exist_ok=True), orient='split'
                )
        
        if history_df is None:
            continue 
            
        print(len(history_df))
        return history_df

        analysis_results = perform_causal_analysis(
            algo_type,
            history_df,
            CAUSAL_PAIRS,
            DIMENSIONS,
            L = 5000,
            ccm_E = 4,
            gc_lag = 4,
            perform_ccm = True,
        )
        
        print(analysis_results)


def perform_causal_analysis(
        policy_name,
        observation_data,
        causal_pairs, 
        dimensions,
        L, 
        ccm_E,
        gc_lag,
        **kwargs):
    # 1. For each fragment length, do causal analysis
    analysis_results = []
    for pair in causal_pairs:
        for dim in dimensions:
            X: pd.Series = observation_data.loc[:, f"{pair[0]}_{dim}"]
            Y: pd.Series = observation_data.loc[:, f"{pair[1]}_{dim}"]

            if len(X) <= L or len(Y) <= L:
                raise Exception("Not Enough data")
            
            X = X[:L]
            Y = Y[:L]
            
            results = list()
            results.extend(get_correlation(X, Y))
            if kwargs.get('perform_ccm', False):
                results.extend(get_ccm(X, Y, ccm_E))
            
            if kwargs.get('perform_granger_linear'):
                results.extend(get_granger_linear(Y, X, gc_lag))
                # results.extend(get_granger_arima(Y, X, gc_lag))

            for result in results:
                analysis_results.append(
                        [
                            (0, policy_name, *pair, result[0], dim),
                            L,
                            result[1],
                        ]
                    )
                
    return analysis_results

history_df = fetch_run_data(nprey, algo_type, rteam)

10000


In [41]:
class spatial_ccm(ccm):
    """
    We're checking causality X -> Y       
    Args
        X: timeseries for variable X that could cause Y
        Y: timeseries for variable Y that could be caused by X
        tau: time lag. default = 1
        E: shadow manifold embedding dimension. default = 2
        L: time period/duration to consider (longer = more data). default = length of X
    """
    def __init__(self, X, Y, sample_index, tau=1, E=2, L=None):
        '''
        X: timeseries for variable X that could cause Y
        Y: timeseries for variable Y that could be caused by X
        tau: time lag
        E: shadow manifold embedding dimension
        L: time period/duration to consider (longer = more data)
        We're checking for X -> Y
        '''
        super().__init__(X, Y, tau, E, L)
        self.My = self.dewdrop_shadow_manifold(Y, sample_index) # shadow manifold for Y (we want to know if info from X is in Y)
        self.t_steps, self.dists = self.get_distances(self.My) # for distances between points in manifold

    def dewdrop_shadow_manifold(self, V, sample_index):
        """
        Given
            V: some time series vector
            tau: lag step
            E: shadow manifold embedding dimension
            L: max time step to consider - 1 (starts from 0)
        Returns
            {t:[t, t-tau, t-2*tau ... t-(E-1)*tau]} = Shadow attractor manifold, dictionary of vectors
        """
        V = V[:self.L] # make sure we cut at L
        sample_index = sample_index[:self.L]
        grouped_df = pd.DataFrame({"V": V, "sample_index": sample_index}).groupby("sample_index")

        M = {t:[] for t in range((self.E-1) * self.tau, self.L)} # shadow manifold
        for name, group in grouped_df:
            L = len(group)
            start_index = group.index[0]
            for t in range((self.E-1) * self.tau, L):
                v_lag = [] # lagged values
                a = t + start_index
                for t2 in range(0, self.E-1 + 1): # get lags, we add 1 to E-1 because we want to include E
                    v_lag.append(V[a-t2*self.tau])
                M[a] = v_lag
            
        filter_M = {k:v for k,v in M.items() if len(v) != 0} # filter out empty vectors
        return filter_M
    
    # get pairwise distances between vectors in the time series
    def get_distances(self, M):
        """
        Args
            M: The shadow manifold from the time series
        Returns
            t_steps: timesteps
            dists: n x n matrix showing distances of each vector at t_step (rows) from other vectors (columns)
        """

        # we extract the time indices and vectors from the manifold M
        # we just want to be safe and convert the dictionary to a tuple (time, vector)
        # to preserve the time inds when we separate them
        t_vec = [(k, v) for k,v in M.items() if len(v) != 0]
        t_steps = np.array([i[0] for i in t_vec])
        vecs = np.array([i[1] for i in t_vec])
        dists = distance.cdist(vecs, vecs)
        return t_steps, dists

    def get_nearest_distances(self, t, t_steps, dists):
        """
        Args:
            t: timestep of vector whose nearest neighbors we want to compute
            t_teps: time steps of all vectors in the manifold M, output of get_distances()
            dists: distance matrix showing distance of each vector (row) from other vectors (columns). output of get_distances()
            E: embedding dimension of shadow manifold M
        Returns:
            nearest_timesteps: array of timesteps of E+1 vectors that are nearest to vector at time t
            nearest_distances: array of distances corresponding to vectors closest to vector at time t
        """
        t_ind = np.where(t_steps == t) # get the index of time t
        dist_t = dists[t_ind].squeeze() # distances from vector at time t (this is one row)

        # get top closest vectors
        nearest_inds = np.argsort(dist_t)[1:self.E+1 + 1] # get indices sorted, we exclude 0 which is distance from itself
        nearest_timesteps = t_steps[nearest_inds] # index column-wise, t_steps are same column and row-wise
        nearest_distances = dist_t[nearest_inds]

        return nearest_timesteps, nearest_distances

    def predict(self, t):
        """
        Args
            t: timestep at manifold of y, My, to predict X at same time step
        Returns
            X_true: the true value of X at time t
            X_hat: the predicted value of X at time t using the manifold My
        """
        eps = 0.000001 # epsilon minimum distance possible
        t_ind = np.where(self.t_steps == t) # get the index of time t
        dist_t = self.dists[t_ind].squeeze() # distances from vector at time t (this is one row)
        nearest_timesteps, nearest_distances = self.get_nearest_distances(t, self.t_steps, self.dists)

        try:
            # get weights
            u = np.exp(-nearest_distances/np.max([eps, nearest_distances[0]])) # we divide by the closest distance to scale
            w = u / np.sum(u)
        except:
            print(nearest_distances)
            print(nearest_timesteps)
            print(t)
            print(self.t_steps)
            print(self.dists)
            raise
        # get prediction of X
        X_true = self.X[t] # get corresponding true X
        X_cor = np.array(self.X)[nearest_timesteps] # get corresponding Y to cluster in Mx
        X_hat = (w * X_cor).sum() # get X_hat

    #     DEBUGGING
    #     will need to check why nearest_distances become nan
    #     if np.isnan(X_hat):
    #         print(nearest_timesteps)
    #         print(nearest_distances)

        return X_true, X_hat

# 1. For each fragment length, do causal analysis
analysis_results = []
for pair in CAUSAL_PAIRS:
    for dim in DIMENSIONS:
        X: pd.Series = history_df.loc[:, f"{pair[0]}_{dim}"]
        Y: pd.Series = history_df.loc[:, f"{pair[1]}_{dim}"]
        sample_index = history_df.loc[:, "episode_num"]

        results = list()

        # 1.1 Correlation
        results.extend(get_correlation(X, Y))

        # 1.2 CCM spatial 
        ccm_spatial = spatial_ccm(X, Y, sample_index, tau=1, E=5, L=5000).causality()
        print(ccm_spatial)


(0.35096982568034873, 3.6128900248898666e-139)
(0.4455174451598273, 7.855699557538382e-233)
(0.08597333708624488, 2.4328276936826416e-09)
(0.16122263105234122, 2.57834783375477e-29)
(0.6639481197614152, 0.0)
(0.6611454064003536, 0.0)
(0.37386703786677733, 4.1740248603429686e-159)
(0.35471525082922506, 2.55794928820229e-142)
(-0.04100654135910868, 0.004490583692504956)
(0.29040859109207856, 6.325464312406909e-94)
(0.5753107458460549, 0.0)
(0.7072192002144684, 0.0)
